In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Raw_Data_Oanda.csv")

In [3]:
df

,datetime,symbol,open,high,low,close,volume,Target
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0
...,...,...,...,...,...,...,...,...
9994,2024-01-19 09:30:00,OANDA:XAUUSD,2022.930,2022.995,2020.870,2020.970,980.0,0
9995,2024-01-19 10:00:00,OANDA:XAUUSD,2020.970,2021.545,2020.790,2021.170,1004.0,0
9996,2024-01-19 10:30:00,OANDA:XAUUSD,2021.170,2021.525,2020.550,2020.615,1238.0,0
9997,2024-01-19 11:00:00,OANDA:XAUUSD,2020.615,2021.935,2020.475,2021.700,2044.0,0


In [4]:
data = df.copy()

In [7]:
def calculate_obv(data):
    obv = [0]

    for i in range(1, len(data)):
        if data['close'][i] > data['close'][i-1]:  
            obv.append(obv[-1] + data['volume'][i])
        elif data['close'][i] < data['close'][i-1]:  
            obv.append(obv[-1] - data['volume'][i])
        else:  
            obv.append(obv[-1])

    return obv

data['OBV'] = calculate_obv(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0


In [8]:
def calculate_rsi(data, period=14):
    delta = data['close'].diff()

    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss

    rsi = 100 - (100 / (1 + rs))

    return rsi

data['RSI'] = calculate_rsi(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0


In [9]:
def calculate_fibonacci_levels(data, period=14):
    rolling_high = data['high'].rolling(window=period, min_periods=1).max()
    rolling_low = data['low'].rolling(window=period, min_periods=1).min()

    data['Fib_23.6%'] = rolling_low + (rolling_high - rolling_low) * 0.236
    data['Fib_38.2%'] = rolling_low + (rolling_high - rolling_low) * 0.382
    data['Fib_50%'] = rolling_low + (rolling_high - rolling_low) * 0.5
    data['Fib_61.8%'] = rolling_low + (rolling_high - rolling_low) * 0.618

    return data

data = calculate_fibonacci_levels(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI,Fib_23.6%,Fib_38.2%,Fib_50%,Fib_61.8%
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN,1910.03286,1910.38107,1910.6625,1910.94393
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0,1910.03286,1910.38107,1910.6625,1910.94393
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0,1910.35736,1910.90632,1911.3500,1911.79368
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0,1910.77272,1911.57864,1912.2300,1912.88136
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0,1910.77272,1911.57864,1912.2300,1912.88136


In [10]:
def calculate_stochastic_oscillator(data, k_period=14, d_period=3):
    low_min = data['low'].rolling(window=k_period).min()
    high_max = data['high'].rolling(window=k_period).max()

    data['%K'] = ((data['close'] - low_min) / (high_max - low_min)) * 100

    data['%D'] = data['%K'].rolling(window=d_period).mean()

    return data

data = calculate_stochastic_oscillator(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI,Fib_23.6%,Fib_38.2%,Fib_50%,Fib_61.8%,%K,%D
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN,1910.03286,1910.38107,1910.6625,1910.94393,NaN,NaN
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0,1910.03286,1910.38107,1910.6625,1910.94393,NaN,NaN
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0,1910.35736,1910.90632,1911.3500,1911.79368,NaN,NaN
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0,1910.77272,1911.57864,1912.2300,1912.88136,NaN,NaN
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0,1910.77272,1911.57864,1912.2300,1912.88136,NaN,NaN


In [11]:
def calculate_parabolic_sar(data):
    data['SAR'] = 0.0
    data['EP'] = 0.0  
    data['AF'] = 0.02  

    # Initial trend
    uptrend = data['close'][1] > data['close'][0]

    # Initial EP
    if uptrend:
        data.at[0, 'EP'] = data['high'][0]
    else:
        data.at[0, 'EP'] = data['low'][0]

    # Initial SAR
    data.at[1, 'SAR'] = data['EP'][0]

    for i in range(2, len(data)):
        if uptrend:
            data.at[i, 'SAR'] = data['SAR'][i - 1] + data['AF'][i - 1] * (data['EP'][i - 1] - data['SAR'][i - 1])
        else:
            data.at[i, 'SAR'] = data['SAR'][i - 1] - data['AF'][i - 1] * (data['SAR'][i - 1] - data['EP'][i - 1])

        if uptrend:
            data.at[i, 'SAR'] = min(data.at[i, 'SAR'], data['low'][i - 1], data['low'][i])
            new_high = data['high'][i] > data['EP'][i - 1]
            data.at[i, 'EP'] = max(data['high'][i], data['EP'][i - 1])
        else:
            data.at[i, 'SAR'] = max(data.at[i, 'SAR'], data['high'][i - 1], data['high'][i])
            new_low = data['low'][i] < data['EP'][i - 1]
            data.at[i, 'EP'] = min(data['low'][i], data['EP'][i - 1])

        if (uptrend and new_high) or (not uptrend and new_low):
            data.at[i, 'AF'] = min(data['AF'][i - 1] + 0.02, 0.2)
        else:
            if (uptrend and data['low'][i] < data['SAR'][i]) or (not uptrend and data['high'][i] > data['SAR'][i]):
                uptrend = not uptrend
                data.at[i, 'SAR'] = data['EP'][i - 1]
                data.at[i, 'AF'] = 0.02
                data.at[i, 'EP'] = data['high'][i] if uptrend else data['low'][i]
            else:
                data.at[i, 'AF'] = data['AF'][i - 1]
                data.at[i, 'EP'] = data['EP'][i - 1]

    return data

data = calculate_parabolic_sar(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI,Fib_23.6%,Fib_38.2%,Fib_50%,Fib_61.8%,%K,%D,SAR,EP,AF
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN,1910.03286,1910.38107,1910.6625,1910.94393,NaN,NaN,0.000000,1911.855,0.02
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0,1910.03286,1910.38107,1910.6625,1910.94393,NaN,NaN,1911.855000,0.000,0.02
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0,1910.35736,1910.90632,1911.3500,1911.79368,NaN,NaN,1873.617900,1913.230,0.04
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0,1910.77272,1911.57864,1912.2300,1912.88136,NaN,NaN,1875.202384,1914.990,0.06
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0,1910.77272,1911.57864,1912.2300,1912.88136,NaN,NaN,1877.589641,1914.990,0.06


In [12]:
def calculate_adx(data, period=14):
    plus_dm = data['high'].diff()
    minus_dm = data['low'].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    minus_dm = minus_dm.abs()

    tr1 = data['high'] - data['low']
    tr2 = (data['high'] - data['close'].shift()).abs()
    tr3 = (data['low'] - data['close'].shift()).abs()
    tr = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)

    smooth_plus_dm = plus_dm.rolling(window=period).sum()
    smooth_minus_dm = minus_dm.rolling(window=period).sum()
    smooth_tr = tr.rolling(window=period).sum()

    data['+DI'] = (smooth_plus_dm / smooth_tr) * 100
    data['-DI'] = (smooth_minus_dm / smooth_tr) * 100

    dx = (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI'])) * 100
    data['ADX'] = dx.rolling(window=period).mean()

    return data

data = calculate_adx(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI,...,Fib_50%,Fib_61.8%,%K,%D,SAR,EP,AF,+DI,-DI,ADX
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN,...,1910.6625,1910.94393,NaN,NaN,0.000000,1911.855,0.02,NaN,NaN,NaN
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0,...,1910.6625,1910.94393,NaN,NaN,1911.855000,0.000,0.02,NaN,NaN,NaN
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0,...,1911.3500,1911.79368,NaN,NaN,1873.617900,1913.230,0.04,NaN,NaN,NaN
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0,...,1912.2300,1912.88136,NaN,NaN,1875.202384,1914.990,0.06,NaN,NaN,NaN
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0,...,1912.2300,1912.88136,NaN,NaN,1877.589641,1914.990,0.06,NaN,NaN,NaN


In [13]:
def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    short_ema = data['close'].ewm(span=short_period, adjust=False).mean()
    long_ema = data['close'].ewm(span=long_period, adjust=False).mean()

    data['MACD'] = short_ema - long_ema

    data['Signal Line'] = data['MACD'].ewm(span=signal_period, adjust=False).mean()

    data['MACD Histogram'] = data['MACD'] - data['Signal Line']

    return data

data = calculate_macd(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI,...,%D,SAR,EP,AF,+DI,-DI,ADX,MACD,Signal Line,MACD Histogram
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN,...,NaN,0.000000,1911.855,0.02,NaN,NaN,NaN,0.000000,0.000000,0.000000
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0,...,NaN,1911.855000,0.000,0.02,NaN,NaN,NaN,0.092536,0.018507,0.074028
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0,...,NaN,1873.617900,1913.230,0.04,NaN,NaN,NaN,0.292414,0.073288,0.219125
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0,...,NaN,1875.202384,1914.990,0.06,NaN,NaN,NaN,0.459641,0.150559,0.309082
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0,...,NaN,1877.589641,1914.990,0.06,NaN,NaN,NaN,0.618926,0.244232,0.374693


In [14]:
def calculate_bollinger_bands(data, period=20):
    data['Middle Band'] = data['close'].rolling(window=period).mean()

    std_dev = data['close'].rolling(window=period).std()

    data['Upper Band'] = data['Middle Band'] + (std_dev * 2)
    data['Lower Band'] = data['Middle Band'] - (std_dev * 2)

    return data

data = calculate_bollinger_bands(data)
data.head()


,datetime,symbol,open,high,low,close,volume,Target,OBV,RSI,...,AF,+DI,-DI,ADX,MACD,Signal Line,MACD Histogram,Middle Band,Upper Band,Lower Band
0,2023-03-16 07:30:00,OANDA:XAUUSD,1911.155,1911.855,1909.470,1910.355,1222.0,0,0.0,NaN,...,0.02,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,2023-03-16 08:00:00,OANDA:XAUUSD,1910.355,1911.670,1909.900,1911.515,1176.0,0,1176.0,100.0,...,0.02,NaN,NaN,NaN,0.092536,0.018507,0.074028,NaN,NaN,NaN
2,2023-03-16 08:30:00,OANDA:XAUUSD,1911.515,1913.230,1910.730,1913.125,1011.0,0,2187.0,100.0,...,0.04,NaN,NaN,NaN,0.292414,0.073288,0.219125,NaN,NaN,NaN
3,2023-03-16 09:00:00,OANDA:XAUUSD,1913.125,1914.990,1912.035,1913.300,978.0,0,3165.0,100.0,...,0.06,NaN,NaN,NaN,0.459641,0.150559,0.309082,NaN,NaN,NaN
4,2023-03-16 09:30:00,OANDA:XAUUSD,1913.300,1914.985,1913.125,1913.720,1035.0,0,4200.0,100.0,...,0.06,NaN,NaN,NaN,0.618926,0.244232,0.374693,NaN,NaN,NaN


In [15]:
data.columns

Index(['datetime', 'symbol', 'open', 'high', 'low', 'close', 'volume',
       'Target', 'OBV', 'RSI', 'Fib_23.6%', 'Fib_38.2%', 'Fib_50%',
       'Fib_61.8%', '%K', '%D', 'SAR', 'EP', 'AF', '+DI', '-DI', 'ADX', 'MACD',
       'Signal Line', 'MACD Histogram', 'Middle Band', 'Upper Band',
       'Lower Band'],
      dtype='object')